In [0]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost
!pip install hyperopt

In [0]:
import pandas as pd
import numpy as np

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

from hyperopt import hp, fmin, tpe, STATUS_OK

import eli5
from eli5.sklearn import PermutationImportance

In [3]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car


In [4]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [0]:
SUFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue
  factorized_value = df[feat].factorize()[0]
  if SUFIX_CAT in feat:
    df[feat] = factorized_value
  else:
    df[feat + SUFIX_CAT] = factorized_value

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split(' ')[0]))
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int( str(x).split('cm')[0].replace(' ', '') ) )

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [14]:
feats = ['param_napęd__cat',
'param_rok-produkcji',
'param_stan__cat',
'param_skrzynia-biegów__cat',
'param_faktura-vat__cat',
'param_moc',
'param_marka-pojazdu__cat',
'feature_kamera-cofania__cat',
'param_typ__cat',
'seller_name__cat',
'param_pojemność-skokowa',
'feature_wspomaganie-kierownicy__cat',
'param_model-pojazdu__cat',
'param_wersja__cat',
'param_kod-silnika__cat',
'feature_system-start-stop__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_czujniki-parkowania-przednie__cat',
'feature_łopatki-zmiany-biegów__cat',
'feature_regulowane-zawieszenie__cat']

xgb_params = {
    'max_depth': 5,
    'n_estimators': 50,
    'learning_rate': 0.1,
    'seed': 0
    }

model = xgb.XGBRegressor(**xgb_params)
run_model( model, feats )

[18:30:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:30:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:30:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9569.227198767323, 72.83561801421891)

In [21]:
def obj_func(params):
  print('Training with params: {}'.format(params))
  mean_mae, scored_std = run_model(xgb.XGBRegressor(**params), feats)
  return {'loss': np.abs(mean_mae), 'status': STATUS_OK }

 # space
xgb_params = {
     'learning_rate': hp.choice('learning_rate', np.arange(0.05, 0.31, 0.05)),
     'max_depth': hp.choice('max_depth', np.arange(5, 16, 1, dtype=int)),
     'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
     'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
     'objective': 'reg:squarederror',
     'n_estimators': 100,
     'seed':0,
 } 

 # run
best = fmin(obj_func, xgb_params, algo=tpe.suggest, max_evals=25)

best

Training with params: {'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.05, 'max_depth': 11, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.55}
Training with params: {'colsample_bytree': 0.5, 'learning_rate': 0.05, 'max_depth': 13, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.75}
Training with params: {'colsample_bytree': 0.8500000000000001, 'learning_rate': 0.05, 'max_depth': 9, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9}
Training with params: {'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.1, 'max_depth': 11, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9500000000000001}
Training with params: {'colsample_bytree': 0.8500000000000001, 'learning_rate': 0.3, 'max_depth': 14, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 1.0}
Training with params: {'colsample_bytree': 0.9500000000000001, 'learnin

{'colsample_bytree': 0.8500000000000001,
 'learning_rate': 1,
 'max_depth': 7,
 'subsample': 0.8500000000000001}

In [0]:
!git 
!git config --global user.email "izabeera@gmail.com"
!git config --global user.name "izabeera"